In [1]:
import BSE
import pandas as pd
import numpy as np
import os

offset_file = 'offset-ibm-1m-170831.csv' 
end_time = 60 * 10 * 2 

n_list = [1, 3, 5, 10]
bid_list = [0.1, 0.3, 0.5, 0.7, 0.9]
ask_list = [10, 20, 30, 50]

results = []

def get_mmm_profit(sess_id):
    profit = 0
    fname = sess_id + '_avg_balance.csv'
    if os.path.exists(fname):
        try:
            df = pd.read_csv(fname, header=None)
            row = df.iloc[-1]
            for i, val in enumerate(row):
                if str(val).startswith('MMM'):
                    profit = float(row[i+1])
                    break
        except:
            pass
        try:
            os.remove(fname)
        except:
            pass
    return profit

print("Starting Grid Search (V2)... This may take a while.")

count = 0
total = len(n_list) * len(bid_list) * len(ask_list)

for n in n_list:
    for bid in bid_list:
        for ask in ask_list:
            count += 1
            if count % 5 == 0:
                print(f"Progress: {count}/{total} configurations tested...")

            n_runs = 2 
            profits = []
            
            for i in range(n_runs):
                sess_id = f"q3_grid_{count}_{i}"
                sellers = [('ZIP', 10)]; buyers = [('ZIP', 10)]
                mrktmakers = [('MMM01', 1, {'n_past_trades': n, 'bid_percent': bid, 'ask_delta': ask})]
                traders = {'sellers': sellers, 'buyers': buyers, 'mrktmakers': mrktmakers}

                try:
                    off = BSE.offset_from_file(offset_file, 1, 2, 100, end_time)
                    sup_range = (100, 200, off)
                    dem_range = (200, 100, off)
                    mode = 'random'
                except:
                    sup_range = (100, 200)
                    dem_range = (200, 100)
                    mode = 'fixed'

                sup_sch = [{'from': 0, 'to': end_time, 'ranges': [sup_range], 'stepmode': mode}]
                dem_sch = [{'from': 0, 'to': end_time, 'ranges': [dem_range], 'stepmode': mode}]
                order_s = {'sup': sup_sch, 'dem': dem_sch, 'interval': 15, 'timemode': 'drip-jitter'}
                
                dump = {'dump_avgbals': True, 'dump_tape': False, 'dump_blotters': False, 'dump_lobs': False, 'dump_strats': False}
                BSE.market_session(sess_id, 0, end_time, traders, order_s, dump, False)
                profits.append(get_mmm_profit(sess_id))
            
            avg_profit = np.mean(profits)
            results.append([n, bid, ask, avg_profit])
            
df_res = pd.DataFrame(results, columns=['n', 'bid', 'ask', 'avg_profit'])
df_res.to_csv('q3_grid_results_temp.csv', index=False)

print("\n--- Search Finished ---")
print("Best Params found:")
best_row = df_res.sort_values('avg_profit', ascending=False).head(1)
print(best_row)

Starting Grid Search (V2)... This may take a while.
MM01 init: n_past_trades=1, bid_percent=0.10000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.10000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.10000, ask_delta=20

MM01 init: n_past_trades=1, bid_percent=0.10000, ask_delta=20

MM01 init: n_past_trades=1, bid_percent=0.10000, ask_delta=30

MM01 init: n_past_trades=1, bid_percent=0.10000, ask_delta=30

MM01 init: n_past_trades=1, bid_percent=0.10000, ask_delta=50

MM01 init: n_past_trades=1, bid_percent=0.10000, ask_delta=50

Progress: 5/80 configurations tested...
MM01 init: n_past_trades=1, bid_percent=0.30000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.30000, ask_delta=10

MM01 init: n_past_trades=1, bid_percent=0.30000, ask_delta=20

MM01 init: n_past_trades=1, bid_percent=0.30000, ask_delta=20

MM01 init: n_past_trades=1, bid_percent=0.30000, ask_delta=30

MM01 init: n_past_trades=1, bid_percent=0.30000, ask_delta=30

MM01 init: n_past_trades=1